In [1]:
import sys, os
sys.path.append(os.path.join(os.getcwd(), '..'))

In [2]:
from dao import fromDB
from backtesting import test
from backtesting import recorder
from strategies import Buyer
from strategies import Seller

In [3]:
import pandas as pd
pd.set_option('display.precision', 2)
pd.__version__

u'0.19.1'

In [4]:
code='300393'
test_df = fromDB.get_stock(code, 'qfq')
test_record = test.single_stock_maxsize(test_df, Buyer.DadBuyer_1, Seller.DadSeller_1, base_period='w', MA5_p=0.99)

In [5]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib import dates as mdates
import mpld3
import numpy as np

In [6]:
gs = GridSpec(3, 1, height_ratios=[1, 4, 1], left=0.05, right=0.95, bottom=0.15, top=0.975, hspace=0)
base_period = test_record.base_period
relevant_df = (test_record.df[[col_name for col_name in test_record.df.columns 
                              if col_name[-1]==base_period]]
               .drop_duplicates()
               .set_index('Date_'+base_period))
fig = plt.figure(figsize=(12,10))
ax_macd = fig.add_subplot(gs[2,0])
ax_close = fig.add_subplot(gs[1,0], sharex=ax_macd)
ax_profit = fig.add_subplot(gs[0, 0], sharex=ax_macd)
plt.setp(ax_close.get_xticklabels(), visible=False);
plt.setp(ax_profit.get_xticklabels(), visible=False);

macd_df = (test_record.df[[col_name for col_name in test_record.df.columns
                           if col_name[-1]=='m']]
           .drop_duplicates()
           .set_index('Date_m'))
ax_macd.bar(macd_df[macd_df.MACD_m>0].index.values, 
            macd_df[macd_df.MACD_m>0].MACD_m.values,
            width=15, color='salmon')
ax_macd.bar(macd_df[macd_df.MACD_m<0].index.values, 
            macd_df[macd_df.MACD_m<0].MACD_m.values,
            width=15, color='steelblue')

ax_close.plot_date(relevant_df.index.values, relevant_df['Close_'+base_period].values, fmt='r-', label='Close')
ax_close.plot_date(relevant_df.index.values, relevant_df['MA_2_'+base_period].values, fmt='b--', lw=2, label='MA_2')
ax_close.plot_date(relevant_df.index.values, relevant_df['MA_5_'+base_period].values, fmt='m:', lw=2, label='MA_5')

profits_dates = np.array(test_record.profits_date)
profits = np.array(test_record.profits)
ax_profit.plot_date(profits_dates[profits>=0], profits[profits>=0], color='r', ms=8)
ax_profit.plot_date(profits_dates[profits<0], profits[profits<0], color='g', ms=8)
ax_close.plot_date(test_record.buy_date, relevant_df['Close_'+base_period].ix[test_record.buy_date].values,
                   '^', ms=12, label='BUY')
ax_close.plot_date(test_record.sell_date, relevant_df['Close_'+base_period].ix[test_record.sell_date].values,
                   'v', ms=12, mfc='salmon', label='SELL')
ax_profit.plot_date([relevant_df.index.values[0], relevant_df.index.values[-1]], [0, 0], 'k--')
for profit_date in profits_dates:
    ax_close.plot_date([profit_date, profit_date], ax_close.get_ylim(), ls='--', color='k', lw=1)
    ax_profit.plot_date([profit_date, profit_date], ax_profit.get_ylim(), ls='--', color='k', lw=1)
ax_macd.grid(True)
ax_close.legend(loc='best');
mpld3.display()

In [30]:
gs = GridSpec(2, 1, height_ratios=[1, 5], left=0.05, right=0.95, bottom=0.15, top=0.975, hspace=0)
base_period = test_record.base_period
relevant_df = (test_record.df[[col_name for col_name in test_record.df.columns 
                              if col_name[-1]==base_period]]
               .drop_duplicates()
               .set_index('Date_'+base_period))
fig = plt.figure(figsize=(12,10))
ax_close = fig.add_subplot(gs[1,0])
ax_profit = fig.add_subplot(gs[0, 0], sharex=ax_close)
plt.setp(ax_profit.get_xticklabels(), visible=False);


ax_close.plot_date(relevant_df.index.values, relevant_df['Close_'+base_period].values, fmt='r-', label='Close')
ax_close.plot_date(relevant_df.index.values, relevant_df['MA_2_'+base_period].values, fmt='b--', lw=2, label='MA_2')
ax_close.plot_date(relevant_df.index.values, relevant_df['MA_5_'+base_period].values, fmt='m:', lw=2, label='MA_5')
ax_close.set_ylim([0, ax_close.get_ylim()[1]])
profits_dates = np.array(test_record.profits_date)
profits = np.array(test_record.profits)
ax_profit.plot_date(profits_dates[profits>=0], profits[profits>=0], color='r', ms=8)
ax_profit.plot_date(profits_dates[profits<0], profits[profits<0], color='g', ms=8)
ax_close.plot_date(test_record.buy_date, relevant_df['Close_'+base_period].ix[test_record.buy_date].values,
                   '^', ms=12, label='BUY')
ax_close.plot_date(test_record.sell_date, relevant_df['Close_'+base_period].ix[test_record.sell_date].values,
                   'v', ms=12, mfc='salmon', label='SELL')
ax_profit.plot_date([relevant_df.index.values[0], relevant_df.index.values[-1]], [0, 0], 'k--')
for profit_date in profits_dates:
    ax_close.plot_date([profit_date, profit_date], ax_close.get_ylim(), ls='--', color='k', lw=1)
    ax_profit.plot_date([profit_date, profit_date], ax_profit.get_ylim(), ls='--', color='k', lw=1)
ax_close.legend(loc='best')

ax_close_pos = ax_close.get_position()
ax_macd_pos = [ax_close_pos.x0, ax_close_pos.y0, ax_close_pos.width, ax_close_pos.height/5.0]
ax_macd = fig.add_axes(ax_macd_pos, frameon=True, sharex=ax_close)
macd_df = (test_record.df[[col_name for col_name in test_record.df.columns
                           if col_name[-1]=='m']]
           .drop_duplicates()
           .set_index('Date_m'))
ax_macd.bar(macd_df[macd_df.MACD_m>0].index.values, 
            macd_df[macd_df.MACD_m>0].MACD_m.values,
            width=15, color='salmon')
ax_macd.bar(macd_df[macd_df.MACD_m<0].index.values, 
            macd_df[macd_df.MACD_m<0].MACD_m.values,
            width=15, color='steelblue')
ax_macd.grid(True)
mpld3.display()

In [37]:
reload(recorder)
reload(test)

<module 'backtesting.test' from '/Users/yugan/Dropbox/personalTrader/resources/../backtesting/test.pyc'>

In [40]:
code='300393'
test_df = fromDB.get_stock(code, 'qfq')
test_record = test.single_stock_maxsize(test_df, Buyer.DadBuyer_1, Seller.DadSeller_1, base_period='w', MA5_p=0.99)
fig, ax_close, ax_profit = test_record.get_plot()

relevant_df = test_record.get_relevant_df()
ax_close.plot_date(relevant_df.index.values, relevant_df['MA_2_'+base_period].values, fmt='b--', lw=2, label='MA_2')
ax_close.plot_date(relevant_df.index.values, relevant_df['MA_5_'+base_period].values, fmt='m:', lw=2, label='MA_5')
ax_close_pos = ax_close.get_position()
ax_macd_pos = [ax_close_pos.x0, ax_close_pos.y0, ax_close_pos.width, ax_close_pos.height/5.0]
ax_macd = fig.add_axes(ax_macd_pos, frameon=True, sharex=ax_close)
macd_df = (test_record.df[[col_name for col_name in test_record.df.columns
                           if col_name[-1]=='m']]
           .drop_duplicates()
           .set_index('Date_m'))
ax_macd.bar(macd_df[macd_df.MACD_m>0].index.values, 
            macd_df[macd_df.MACD_m>0].MACD_m.values,
            width=15, color='salmon')
ax_macd.bar(macd_df[macd_df.MACD_m<0].index.values, 
            macd_df[macd_df.MACD_m<0].MACD_m.values,
            width=15, color='steelblue')
ax_macd.grid(True)
mpld3.display()